In [44]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 3: Lobby ---</h2><p>You descend a short staircase, enter the surprisingly vast lobby, and are quickly cleared by the security checkpoint. When you get to the main elevators, however, you discover that each one has a red light above it: they're all <em>offline</em>.</p>
<p>"Sorry about that," an Elf apologizes as she tinkers with a nearby control panel. "Some kind of electrical surge seems to have fried them. I'll try to get them online soon."</p>
<p>You explain your need to get further underground. "Well, you could at least take the escalator down to the printing department, not that you'd get much further than that without the elevators working. That is, you could if the escalator weren't also <span title="Escalator temporarily stairs. Sorry for the convenience.">offline</span>."</p>
<p>"But, don't worry! It's not fried; it just needs power. Maybe you can get it running while I keep working on the elevators."</p>
<p>There are batteries nearby that can supply emergency power to the escalator for just such an occasion. The batteries are each labeled with their <a href="/2020/day/10">joltage</a> rating, a value from <code>1</code> to <code>9</code>. You make a note of their joltage ratings (your puzzle input). For example:</p>
<pre><code>987654321111111
811111111111119
234234234234278
818181911112111
</code></pre>
<p>The batteries are arranged into <em>banks</em>; each line of digits in your input corresponds to a single bank of batteries. Within each bank, you need to turn on <em>exactly two</em> batteries; the joltage that the bank produces is equal to the number formed by the digits on the batteries you've turned on. For example, if you have a bank like <code>12345</code> and you turn on batteries <code>2</code> and <code>4</code>, the bank would produce <code>24</code> jolts. (You cannot rearrange batteries.)</p>
<p>You'll need to find the largest possible joltage each bank can produce. In the above example:</p>
<ul>
<li>In <code><em>98</em>7654321111111</code>, you can make the largest joltage possible, <em><code>98</code></em>, by turning on the first two batteries.</li>
<li>In <code><em>8</em>1111111111111<em>9</em></code>, you can make the largest joltage possible by turning on the batteries labeled <code>8</code> and <code>9</code>, producing <em><code>89</code></em> jolts.</li>
<li>In <code>2342342342342<em>78</em></code>, you can make <em><code>78</code></em> by turning on the last two batteries (marked <code>7</code> and <code>8</code>).</li>
<li>In <code>818181<em>9</em>1111<em>2</em>111</code>, the largest joltage you can produce is <em><code>92</code></em>.</li>
</ul>
<p>The total output joltage is the sum of the maximum joltage from each bank, so in this example, the total output joltage is <code>98</code> + <code>89</code> + <code>78</code> + <code>92</code> = <code><em>357</em></code>.</p>
<p>There are many batteries in front of you. Find the maximum joltage possible from each bank; <em>what is the total output joltage?</em></p>
</article>


In [45]:
tests = [
    {
        "name": "Example",
        "s": """
            987654321111111
            811111111111119
            234234234234278
            818181911112111
        """,
        "expected": 357,
    },
]


@test(tests=tests)
def part_I(s: str) -> int:
    return sum(
        max(
            int(f"{battery[i]}{battery[j]}")
            for i in range(len(battery))
            for j in range(i + 1, len(battery))
        )
        for battery in re.findall(r"\d+", s)
    )


Test Example passed, for part_I.
Success


In [46]:
with open("../input/day3.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 17412


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>17412</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The escalator doesn't move. The Elf explains that it probably needs more joltage to overcome the <a href="https://en.wikipedia.org/wiki/Static_friction" target="_blank">static friction</a> of the system and hits the big red "joltage limit safety override" button. You lose count of the number of times she needs to confirm "yes, I'm sure" and decorate the lobby a bit while you wait.</p>
<p>Now, you need to make the largest joltage by turning on <em>exactly twelve</em> batteries within each bank.</p>
<p>The joltage output for the bank is still the number formed by the digits of the batteries you've turned on; the only difference is that now there will be <code><em>12</em></code> digits in each bank's joltage output instead of two.</p>
<p>Consider again the example from before:</p>
<pre><code>987654321111111
811111111111119
234234234234278
818181911112111
</code></pre>
<p>Now, the joltages are much larger:</p>
<ul>
<li>In <code><em>987654321111</em>111</code>, the largest joltage can be found by turning on everything except some <code>1</code>s at the end to produce <code><em>987654321111</em></code>.</li>
<li>In the digit sequence <code><em>81111111111</em>111<em>9</em></code>, the largest joltage can be found by turning on everything except some <code>1</code>s, producing <code><em>811111111119</em></code>.</li>
<li>In <code>23<em>4</em>2<em>34234234278</em></code>, the largest joltage can be found by turning on everything except a <code>2</code> battery, a <code>3</code> battery, and another <code>2</code> battery near the start to produce <code><em>434234234278</em></code>.</li>
<li>In <code><em>8</em>1<em>8</em>1<em>8</em>1<em>911112111</em></code>, the joltage <code><em>888911112111</em></code> is produced by turning on everything except some <code>1</code>s near the front.</li>
</ul>
<p>The total output joltage is now much larger: <code>987654321111</code> + <code>811111111119</code> + <code>434234234278</code> + <code>888911112111</code> = <code><em>3121910778619</em></code>.</p>
<p><em>What is the new total output joltage?</em></p>
</article>


In [49]:
import heapq

from more_itertools import first


tests = [
    {
        "name": "Example",
        "s": """
            987654321111111
            811111111111119
            234234234234278
            818181911112111
        """,
        "expected": 3121910778619,
    },
]


def biggest_number(battery: list[int], size=12) -> int:
    numbers_indici = [[] for _ in range(10)]
    max_n = -inf
    for i, n in enumerate(battery):
        numbers_indici[n].append(i)
        max_n = max(max_n, n)

    buffer = []
    last_index = -1

    n = len(battery)
    while size > 0:
        i = max_n
        while (
            not numbers_indici[i]
            or n - numbers_indici[i][0] < size
            or numbers_indici[i][0] <= last_index
        ):
            if numbers_indici[i] and numbers_indici[i][0] <= last_index:
                numbers_indici[i].pop(0)
            else:
                i -= 1

        buffer.append(i)
        last_index = numbers_indici[i].pop(0)
        size -= 1

    return int("".join(map(str, buffer)))


@test(tests=tests)
def part_II(s: str) -> int:
    som = 0
    for battery in map(lambda l: list(map(int, l)), re.findall(r"\d+", s)):
        som += biggest_number(battery)

    return som


Test Example passed, for part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [50]:
print(f"Part II: {part_II(puzzle)}")

Part II: 172681562473501


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>172681562473501</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
